# SageMaker Studio Notebooks と SageMaker Training の環境を同一にする

In [ ]:
import boto3
import sagemaker
from time import sleep
region = boto3.session.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
ecr = boto3.client('ecr')
sm_client = boto3.client('sagemaker')
REPOSITORY_NAME = 'sagemaker-studio-custom'
IMAGE_NAME = 'custom-py310-buster-training'
KERNEL_NAME = 'python3'
DISPLAY_NAME = 'Python 3'
role = sagemaker.get_execution_role()
print(region)
print(account)
print(role)

In [ ]:
!aws ecr delete-repository --repository-name {REPOSITORY_NAME} --force
!aws ecr create-repository --repository-name {REPOSITORY_NAME}

In [ ]:
%%writefile requirements.txt
ipykernel==6.20.1
numpy==1.22.3
sagemaker==2.144.0
sagemaker-training==4.4.8

In [ ]:
%%writefile Dockerfile
FROM python:3.10.10-buster

COPY requirements.txt /root/requirements.txt
COPY Dockerfile /root/Dockerfile

RUN pip install -r /root/requirements.txt && \
        python -m ipykernel install --sys-prefix

In [ ]:
!docker build . -t {IMAGE_NAME} -t {account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME}

In [ ]:
!$(aws ecr get-login --region {region} --registry-ids {account} --no-include-email)
!docker push {account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME}

In [ ]:
sm_client.delete_image(
    ImageName=IMAGE_NAME,
)

In [ ]:
sm_client.create_image(
    ImageName=IMAGE_NAME,
    RoleArn=role,
)

In [ ]:
sleep(1)
sm_client.create_image_version(
    BaseImage=f'{account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME}',
    ImageName=IMAGE_NAME,
)

In [ ]:
# check
sm_client.describe_image_version(
    ImageName=IMAGE_NAME
)

In [ ]:
output = !docker run {account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME} id -u
default_uid = int(output[0])
output = !docker run {account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME} id -g
default_gid = int(output[0])
print(default_uid, default_gid)

In [ ]:
sm_client.create_app_image_config(
    AppImageConfigName=IMAGE_NAME,
    KernelGatewayImageConfig={
        'KernelSpecs': [
            {
                'Name': KERNEL_NAME,
                'DisplayName': DISPLAY_NAME
            },
        ],
        'FileSystemConfig': {
            'MountPath': '/root/data',
            'DefaultUid': default_uid,
            'DefaultGid': default_gid
        }
    }
)

In [ ]:
# 例えば
domain_id = sm_client.list_domains()['Domains'][0]['DomainId']

In [ ]:
sm_client.update_domain(
    DomainId = domain_id,
    DefaultUserSettings = {
        'KernelGatewayAppSettings' : {
            'CustomImages' : [
                {
                    'ImageName' : IMAGE_NAME,
                    'AppImageConfigName' : IMAGE_NAME
                }
            ]
        }
    }
)

In [ ]:
sm_client.describe_domain(
    DomainId = domain_id
)